In [1]:
import arrow
import socket
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

from april import Evaluator
from april.anomalydetection import *
from april.database import EventLog
from april.database import Model
from april.database import get_engine
from april.dataset import Dataset
from april.fs import DATE_FORMAT
from april.fs import get_event_log_files


In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU found")
    print("Memory growth set")
else:
    print("No GPU found")

2025-03-20 13:24:59.269339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742473499.788870   38694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742473499.929683   38694 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742473501.209972   38694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742473501.210004   38694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742473501.210006   38694 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU found
Memory growth set


In [3]:
import ltn

This is the train method that can be called in parallel.

In [3]:
def fit_and_save(dataset_name, ad, ad_kwargs=None, fit_kwargs=None):
    if ad_kwargs is None:
        ad_kwargs = {}
    if fit_kwargs is None:
        fit_kwargs = {}

    # Save start time
    start_time = arrow.now()

    # Dataset
    dataset = Dataset(dataset_name)

    # AD
    ad = ad(**ad_kwargs)

    # Train and save
    ad.fit(dataset, **fit_kwargs)
    file_name = f'{dataset_name}_{ad.abbreviation}_{start_time.format(DATE_FORMAT)}'
    model_file = ad.save(file_name)

    # Save end time
    end_time = arrow.now()

    # Cache result
    Evaluator(model_file.str_path).cache_result()

    # Calculate training time in seconds
    training_time = (end_time - start_time).total_seconds()

    # Write to database
    engine = get_engine()
    session = Session(engine)

    session.add(Model(creation_date=end_time.datetime,
                      algorithm=ad.name,
                      training_duration=training_time,
                      file_name=model_file.file,
                      training_event_log_id=EventLog.get_id_by_name(dataset_name),
                      training_host=socket.gethostname(),
                      hyperparameters=str(dict(**ad_kwargs, **fit_kwargs))))
    session.commit()
    session.close()

    if isinstance(ad, NNAnomalyDetector):
        from keras.backend import clear_session
        clear_session()

Now, we can run the training for the anomaly detection methods.

In [4]:
datasets = sorted([e.name for e in get_event_log_files() if e.p == 0.3])
select_datasets = ["paper", "p2p", "small", "medium"]
select_attributes = ["-1", "-2"]
datasets = [d for d in datasets if any([s in d for s in select_datasets])]
datasets = [d for d in datasets if any([s in d for s in select_attributes])]
print(datasets)


['medium-0.3-1', 'medium-0.3-2', 'p2p-0.3-1', 'p2p-0.3-2', 'paper-0.3-1', 'small-0.3-1', 'small-0.3-2']


In [5]:
ads = [
    dict(ad=RandomAnomalyDetector),
    # dict(ad=TStidePlus, ad_kwargs=dict(k=2)),
    # dict(ad=OneClassSVM),
    # dict(ad=LikelihoodPlusAnomalyDetector),
    # dict(ad=BoehmerLikelihoodAnomalyDetector),
    dict(ad=NaiveAnomalyDetector),
    # dict(ad=NaivePlusAnomalyDetector),
    dict(ad=SamplingAnomalyDetector),
    dict(ad=DAE, fit_kwargs=dict(epochs=50, batch_size=500)),
    # dict(ad=BINetv1, fit_kwargs=dict(epochs=20, batch_size=500)),
    # dict(ad=BINetv2, fit_kwargs=dict(epochs=20, batch_size=500)),
    # dict(ad=BINetv3, fit_kwargs=dict(epochs=20, batch_size=500))
]
for ad in ads:
    [fit_and_save(d, **ad) for d in tqdm(datasets, desc=ad['ad'].name, leave=True, position=1)]

Random:   0%|          | 0/7 [00:00<?, ?it/s]

Naive:   0%|          | 0/7 [00:00<?, ?it/s]

Sampling:   0%|          | 0/7 [00:00<?, ?it/s]

DAE:   0%|          | 0/7 [00:00<?, ?it/s]

I0000 00:00:1742386493.727817   22474 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 839ms/step - loss: 0.2486 - val_loss: 0.2432
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2389 - val_loss: 0.2292
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2206 - val_loss: 0.2004
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1862 - val_loss: 0.1490
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1316 - val_loss: 0.0817
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0696 - val_loss: 0.0309
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0276 - val_loss: 0.0121
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0122 - val_loss: 0.0077
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0081 - val_loss: 0.0067
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0071 - val_loss: 0.0064
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0067 - val_loss: 0.0062
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0065 - val_loss: 0.0061
Epoch 13/50

2025-03-19 13:17:30.237258: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 236 bytes spill stores, 236 bytes spill loads

2025-03-19 13:17:30.280914: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 80 bytes spill stores, 80 bytes spill loads

2025-03-19 13:17:30.302775: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16_0', 16 bytes spill stores, 16 bytes spill loads

2025-03-19 13:17:30.744289: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 620 bytes spill stores, 620 bytes spill loads

2025-03-19 13:17:30.840181: I external/loc

147/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

2025-03-19 13:17:34.868838: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 248 bytes spill stores, 248 bytes spill loads

2025-03-19 13:17:34.892082: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 80 bytes spill stores, 80 bytes spill loads

2025-03-19 13:17:35.039567: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 1100 bytes spill stores, 1200 bytes spill loads

2025-03-19 13:17:35.659292: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 80 bytes spill stores, 80 bytes spill loads

2025-03-19 13:17:35.804231: I external/loc

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step


In [ ]:
print("EOF")

EOF


: 